In [6]:
import pandas as pd
import os
current_path = os.getcwd()
bug_file = os.path.join(current_path, 'data','bugs.xlsx')
regression_bugs = pd.read_excel(bug_file,sheet_name='0709合并')

# fastjson_data = regression_bugs[regression_bugs['proj.'].str.contains('fastjson', na=False)]
#
# id_data = fastjson_data['ID'].tolist()

bug_data = regression_bugs
id_data = bug_data['ID'].tolist()


from sql_manager import MySQLDatabase
db = MySQLDatabase("10.176.34.95", "root", "1235", "code_annotation2")
db.connect()


ret = pd.DataFrame(columns=['project_name','commit_hash'])
results = []
for id in id_data:
    sql = "SELECT project_name, bic as commit_hash FROM regressions_all " # WHERE project_name = 'alibaba_fastjson'
    sql += " AND id = " + str(id)
    print(sql)
    result_item = db.fetch_query(sql)
    results.extend(result_item)


# 如果 results 不为空，将它们转换为 DataFrame
if results:
    ret = pd.DataFrame(results, columns=['project_name', 'commit_hash'])

print(len(ret))

/Users/zhjlu/opt/anaconda3/envs/Django/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Connected to MySQL database
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 1336
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 400
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 75
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 589
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 497
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 497
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 968
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 1693
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 295
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 1749
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 484
SELECT project_name, bic as commit_hash FROM regressions_all WHERE id = 1240
SELECT project_name, bic as commit_hash FROM regressions_

In [26]:
ret.head(10)
# 对 'project_name' 列中的下划线进行分割并只保留后半部分
ret['project_name'] = ret['project_name'].str.split('_').str[-1]
ret.to_pickle('../data/fastjson_regression_bugs.pkl') # 待处理数据

,project_name,commit_hash
0,cron-utils,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44
1,cactoos,7775a67e7a6ce47f8a6a54309a21221147cb0328
2,fastjson,c929dfe9bbebaf28315b161c8d34846ea922062e
3,fastjson,7cc1a5e8fd3d021f970900a01b51442699ce36c4
4,java-util,20cbc65e6dce821d08efa9620f455020635013da
...,...,...
275,fastjson,6326029f04de93af06465090199eb045d7b8eaf8
276,vectorz,115bd172d7a310f16dea9ca465b9c41a5140f8b0
277,jsoup,8391b7231a2519557d41209d728acc4f422af381
278,fastjson,03cd1676a16d00c5e9001379f73e20975516e8cc


JITFIne自己的专家特征提取脚本

接下来调用test.py 提取专家特征

In [35]:
import pickle
current_path = os.getcwd()
changes_example_path = os.path.join(current_path, 'data','changes_test.pkl')
changes_example = pickle.load(open(changes_example_path,'rb'))
type(changes_example)


list

In [37]:
changes_example[3][0].keys()

dict_keys(['added_code', 'removed_code'])

In [38]:
type(changes_example[3][1]['added_code'])

set

In [43]:
current_path = os.getcwd()
expert_features_path = os.path.join(current_path, 'data','metrics.pkl') #merge数据没处理
expert_features = pickle.load(open(expert_features_path,'rb'))
type(expert_features)
expert_features['is_buggy_commit'] = 1
print(expert_features.columns)
manual_features_columns = ['la', 'ld', 'nf', 'ns', 'nd', 'entropy', 'ndev',
                           'lt', 'nuc', 'age', 'exp', 'rexp', 'sexp']
expert_features[manual_features_columns] = expert_features[manual_features_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
expert_features.to_pickle('../data/features_reg_test.pkl')

Index(['project', 'parent_hashes', 'commit_hash', 'author_name',
       'author_email', 'author_date', 'author_date_unix_timestamp',
       'commit_message', 'la', 'ld', 'fileschanged', 'nf', 'ns', 'nd',
       'entropy', 'ndev', 'lt', 'nuc', 'age', 'exp', 'rexp', 'sexp',
       'classification', 'fix', 'is_buggy_commit'],
      dtype='object')


In [44]:

expert_features['fix'] = expert_features['fix'].apply(lambda x: float(bool(x)))
df = expert_features.astype({i: 'float32' for i in manual_features_columns})

233

In [45]:
expert_features_example = pickle.load(open(os.path.join(current_path,'data','features_test.pkl') ,'rb'))

In [46]:
expert_features_example.columns == expert_features.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [47]:
import subprocess
import re
from pathlib import Path
import os
current_path = os.getcwd()
repos_data_path = os.path.join(Path(current_path).parent, 'repos')

commits = []
buggy_labels = []
msgs = []
commits_code_changes = []
changes_results = []

for index, column in expert_features.iterrows():
    project_name = column['project']
    commit_hash = column['commit_hash']
    commits.append(commit_hash)
    buggy_labels.append(1)
    msgs.append(column['commit_message'])
    project_path = os.path.join(repos_data_path, project_name)
    short_commit_hash = commit_hash[:8]
    diff_path = os.path.join(project_path, short_commit_hash+ '.txt')
    diffs =''
    if os.path.exists(diff_path):
        diffs = open(diff_path, 'r').read()
    else:
        command = f"git show {commit_hash}"
        # 打开文件 commit_hash.txt 以写入模式
        with open(diff_path, 'w') as file:
            # 使用 subprocess.run 执行命令，将输出重定向到文件中
            subprocess.run(command, shell=True, stdout=file, stderr=subprocess.PIPE,text=True, cwd=project_path)
        # 读取文件内容
        with open(diff_path, 'r', encoding='utf-8') as file:
            diffs = file.read()
    code_changes = {
        'added_code': set(),
        'removed_code': set()
    }
    for match in re.finditer(r'diff --git a/(.*?) b/(.*?)\nindex.*?\n--- a/.*?\n\+\+\+ b/(.*?)\n((?:.*\n)+?)(?=\ndiff --git|\Z)', diffs, re.S):
        changes = match.group(4)
        for line in changes.split('\n'):
            if line.startswith('+') and not line.startswith('+++'):
                code_line = line[1:].strip()
                is_comments =  code_line.startswith('//') or code_line.startswith('/*') or code_line.startswith('*') or not code_line 
                if not is_comments:
                    code_changes['added_code'].add(code_line)

            elif line.startswith('-') and not line.startswith('---'):
                code_line = line[1:].strip()
                is_comments =  code_line.startswith('//') or code_line.startswith('/*') or code_line.startswith('*') or not code_line
                if not is_comments:
                    code_changes['removed_code'].add(code_line)
    commits_code_changes.append(code_changes)
changes_results.append(commits)
changes_results.append(buggy_labels)
changes_results.append(msgs)
changes_results.append(commits_code_changes)
print(len(commits))
print(len(buggy_labels))
print(len(msgs))
print(len(commits_code_changes))
print(len(changes_results))
print(type(changes_results))   
print(type(changes_results[0]))
print(type(changes_results[0][1]))



233
233
233
233
4
<class 'list'>
<class 'list'>
<class 'str'>


In [17]:
changes_results[3][0]

{'added_code': {"&& ch == '['",
  "&& path.charAt(pos + 1) == ']'",
  "&& path.charAt(pos + 2) == '.') {",
  "&& path.charAt(pos) == '*'",
  ', "$..[*].foo.bar");',
  ', "$..foo");',
  ', "$..foo.bar");',
  'JSON.parseObject("{\\"x\\": {\\"foo\\": {\\"bar\\": 4}}, \\"y\\": {\\"foo\\": 1}}")',
  'JSON.parseObject("{\\"x\\": {\\"foo\\": {\\"baz\\": 4}}, \\"y\\": {\\"foo\\": 1}}")',
  'Object fieldValue = path.getPropertyValue(currentObject, propertyNames[i], propertyNamesHash[i]);',
  'Object itemValue = getPropertyValue(obj, propertyName, propertyNameHash);',
  'Object propertyValue = path.getPropertyValue(item, propertyName, propertyNameHash);',
  'Object result = JSONPath.eval(',
  'assertEquals(1, ((List) result).size());',
  'assertEquals(2, ((List) result).size());',
  'assertEquals(4, ((List) result).get(0));',
  'assertTrue(((List) result).isEmpty());',
  'if (path.length() > pos + 3',
  'import com.alibaba.fastjson.JSON;',
  'import com.alibaba.fastjson.JSONPath;',
  'import jav

In [18]:
changes_example[3][3]

{'added_code': {'+ " Parent = " + parentModRevID ) ;',
  'Map . Entry prop = ( Map . Entry ) iter . next ( ) ;',
  'Map parentPomProps = PomModuleDescriptorBuilder . extractPomProperties ( parentDescr . getExtraInfo ( ) ) ;',
  'Message . warn ( "impossible to load parent for " + descriptorURL + " . "',
  'ModuleDescriptor parentDescr = null ;',
  'ModuleRevisionId parentModRevID = ModuleRevisionId . newInstance (',
  'PomReader domReader = new PomReader ( descriptorURL , res ) ;',
  'ResolvedModuleRevision parentModule = parseOtherPom ( ivySettings ,',
  'String pluginExtraInfo = ( String ) ivyModuleDescriptor . getExtraInfo ( ) . get ( "m : maven . plugins" ) ;',
  'domReader . getParentArtifactId ( ) ,',
  'domReader . getParentGroupId ( ) ,',
  'domReader . getParentVersion ( ) ) ;',
  'domReader . setProperty ( ( String ) prop . getKey ( ) , ( String ) prop . getValue ( ) ) ;',
  'for ( Iterator iter = parentPomProps . entrySet ( ) . iterator ( ) ; iter . hasNext ( ) ; ) {',
  'if

In [51]:

pickle.dump(changes_results, open('../data/changes_test.pkl', 'wb'))

In [52]:
test_buggy_commit_lines_df = pd.read_pickle(os.path.join(current_path, 'data','test_buggy_commit_lines_df.pkl'))
columns = test_buggy_commit_lines_df.columns
test_buggy_reg_lines_results = pd.DataFrame(columns=columns)
test_buggy_reg_lines_results.head(10)

,project,file,change type,commit hash,idx,code line,label


In [53]:
print(columns)

Index(['project', 'file', 'change type', 'commit hash', 'idx', 'code line',
       'label'],
      dtype='object')


In [54]:

for index, column in expert_features.iterrows():
    project_name = column['project']
    commit_hash = column['commit_hash']

    project_path = os.path.join(repos_data_path, project_name)
    short_commit_hash = commit_hash[:8]
    diff_path = os.path.join(project_path, short_commit_hash+ '.txt')
    diffs =''
    if os.path.exists(diff_path):
        diffs = open(diff_path, 'r').read()
    else:
        command = f"git show {commit_hash}"
        # 打开文件 commit_hash.txt 以写入模式
        with open(diff_path, 'w') as file:
            # 使用 subprocess.run 执行命令，将输出重定向到文件中
            subprocess.run(command, shell=True, stdout=file, stderr=subprocess.PIPE,text=True, cwd=project_path)
        # 读取文件内容
        with open(diff_path, 'r', encoding='utf-8') as file:
            diffs = file.read()

    added_lines = []  # 使用 set 存储 added 行
    deleted_lines = []  # 使用 set 存储 deleted 行        
    seen_lines = set() 

    for match in re.finditer(r'diff --git a/(.*?) b/(.*?)\nindex.*?\n--- a/.*?\n\+\+\+ b/(.*?)\n((?:.*\n)+?)(?=\ndiff --git|\Z)', diffs, re.S):
        file_path = match.group(1)
        changes = match.group(4)
        rowidx = 0  # 初始化 rowidx 为 0

        # 遍历所有的行并分类
        for line in changes.split('\n'):
            if line.startswith('+') and not line.startswith('+++'):
                code_line = line[1:].strip()  # 去掉 '+'
                is_comments = code_line.startswith('//') or code_line.startswith('/*') or code_line.startswith('*') or not code_line
                if not is_comments and code_line not in seen_lines:
                    row_item = {'project': project_name, 'file': file_path, 'change type': 'added', 'commit hash': commit_hash, 'idx': rowidx, 'code line': code_line, 'label': 1}
                    added_lines.append(row_item)
                    seen_lines.add(code_line)
                    rowidx += 1  # 每次添加后递增 rowidx

        # 在继续递增 rowidx 的基础上处理 deleted 行
        for line in changes.split('\n'):
            if line.startswith('-') and not line.startswith('---'):
                code_line = line[1:].strip()  # 去掉 '-'
                is_comments = code_line.startswith('//') or code_line.startswith('/*') or code_line.startswith('*') or not code_line
                if not is_comments and code_line not in seen_lines:
                    row_item = {'project': project_name, 'file': file_path, 'change type': 'deleted', 'commit hash': commit_hash, 'idx': rowidx, 'code line': code_line, 'label': 1}
                    deleted_lines.append(row_item)
                    seen_lines.add(code_line)
                    rowidx += 1  # 每次添加后递增 rowidx

    # 将 added 行和 deleted 行转换为 DataFrame
    added_df = pd.DataFrame(added_lines)
    deleted_df = pd.DataFrame(deleted_lines)

    # 确保 added 行在 deleted 行之前，合并 DataFrame
    test_buggy_reg_lines_results = pd.concat([test_buggy_reg_lines_results, added_df, deleted_df], ignore_index=True)


In [55]:
test_buggy_reg_lines_results

,project,file,change type,commit hash,idx,code line,label
0,cron-utils,src/main/java/com/cronutils/builder/CronBuilde...,added,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44,0,expression.accept(new ValidationFieldExpressio...,1
1,cron-utils,src/main/java/com/cronutils/builder/CronBuilde...,added,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44,1,new ValidationFieldExpressionVisitor(getCronDe...,1
2,cron-utils,src/main/java/com/cronutils/builder/CronBuilde...,added,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44,2,public CronDefinition(final List<FieldDefiniti...,1
3,cron-utils,src/main/java/com/cronutils/builder/CronBuilde...,added,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44,3,"return new CronDefinition(values, validations,...",1
4,cron-utils,src/main/java/com/cronutils/builder/CronBuilde...,added,ff67527e69868a2c2b05ab8a1ddcca8d8f896e44,4,.withMinutes().withStrictRange().and(),1
...,...,...,...,...,...,...,...
25862,memoryfilesystem,src/main/java/com/github/marschall/memoryfiles...,added,c14b3677fb45429018e21fff5016abd9e15dc922,10,PosixFileAttributeView unsupportedView = Files...,1
25863,memoryfilesystem,src/main/java/com/github/marschall/memoryfiles...,added,c14b3677fb45429018e21fff5016abd9e15dc922,11,assertNull(unsupportedView);,1
25864,memoryfilesystem,src/main/java/com/github/marschall/memoryfiles...,deleted,c14b3677fb45429018e21fff5016abd9e15dc922,12,<V extends FileAttributeView> V getLazyFileAtt...,1
25865,memoryfilesystem,src/main/java/com/github/marschall/memoryfiles...,deleted,c14b3677fb45429018e21fff5016abd9e15dc922,13,return memoryFileSystem.getLazyFileAttributeVi...,1


In [56]:
# 获取 'commit hash' 列的不重复数量
unique_commit_hash_count = test_buggy_reg_lines_results['commit hash'].nunique()
print(unique_commit_hash_count)

for index,column in test_buggy_reg_lines_results.iterrows():
    commit_hash = column['commit hash']
    project_name = column['project']
    file_path = column['file']
    change_type = column['change type']
    idx = column['idx']
    code_line = column['code line']
    label = column['label']
    print(f'commit hash: {commit_hash}, project: {project_name}, file: {file_path}, change type: {change_type}, idx: {idx}, code line: {code_line}, label: {label}')

232
commit hash: ff67527e69868a2c2b05ab8a1ddcca8d8f896e44, project: cron-utils, file: src/main/java/com/cronutils/builder/CronBuilder.java, change type: added, idx: 0, code line: expression.accept(new ValidationFieldExpressionVisitor(constraints));, label: 1
commit hash: ff67527e69868a2c2b05ab8a1ddcca8d8f896e44, project: cron-utils, file: src/main/java/com/cronutils/builder/CronBuilder.java, change type: added, idx: 1, code line: new ValidationFieldExpressionVisitor(getCronDefinition().getFieldDefinition(fieldName).getConstraints()), label: 1
commit hash: ff67527e69868a2c2b05ab8a1ddcca8d8f896e44, project: cron-utils, file: src/main/java/com/cronutils/builder/CronBuilder.java, change type: added, idx: 2, code line: public CronDefinition(final List<FieldDefinition> fieldDefinitions, final Set<CronConstraint> cronConstraints, final boolean matchDayOfWeekAndDayOfMonth) {, label: 1
commit hash: ff67527e69868a2c2b05ab8a1ddcca8d8f896e44, project: cron-utils, file: src/main/java/com/cronutils/

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
test_buggy_reg_lines_results.to_pickle('../data/test_buggy_reg_lines_results.pkl')